In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MOVIELENS 25M.com').getOrCreate()

In [ ]:
!ls


movies.csv  ratings.csv  sample_data


In [ ]:
ratings = spark.read.csv(
    "/content/ratings.csv",
    header=True,
    inferSchema=True
)


In [ ]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:
ratings.count()

453712

In [ ]:
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
+------+-------+------+----------+
only showing top 5 rows


In [ ]:
movies = spark.read.csv(
    "/content/movies.csv",
    header=True,
    inferSchema=True
)


In [ ]:
movies.show(5)


+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows


In [ ]:
ratings.describe().show()


+-------+------------------+------------------+------------------+--------------------+
|summary|            userId|           movieId|            rating|           timestamp|
+-------+------------------+------------------+------------------+--------------------+
|  count|            453712|            453712|            453712|              453712|
|   mean|1589.6495904891208|20462.151069400854|3.5517200338540746|1.2068150287334123E9|
| stddev| 887.1374704779284| 38184.54677824372|1.0527163425839643|2.2922042230072683E8|
|    min|                 1|                 1|               0.5|           789652009|
|    max|              3128|            208939|               5.0|          1574253766|
+-------+------------------+------------------+------------------+--------------------+



In [ ]:
ratings_clean = ratings.dropna()
movies_clean = movies.dropna()


In [ ]:
ratings_movies = ratings_clean.join(
    movies_clean,
    on="movieId",
    how="inner"
)


In [ ]:
ratings_movies.printSchema()
ratings_movies.show(5)


root
 |-- movieId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+------+------+----------+--------------------+--------------------+
|movieId|userId|rating| timestamp|               title|              genres|
+-------+------+------+----------+--------------------+--------------------+
|    296|     1|   5.0|1147880044| Pulp Fiction (1994)|Comedy|Crime|Dram...|
|    306|     1|   3.5|1147868817|Three Colors: Red...|               Drama|
|    307|     1|   5.0|1147868828|Three Colors: Blu...|               Drama|
|    665|     1|   5.0|1147878820|  Underground (1995)|    Comedy|Drama|War|
|    899|     1|   3.5|1147868510|Singin' in the Ra...|Comedy|Musical|Ro...|
+-------+------+------+----------+--------------------+--------------------+
only showing top 5 rows


In [ ]:
ratings_movies.count()


453712

In [ ]:
from pyspark.sql.functions import count

top_rated_movies = ratings_movies.groupBy("title") \
    .agg(count("rating").alias("total_ratings")) \
    .orderBy("total_ratings", ascending=False)

top_rated_movies.show(10)


+--------------------+-------------+
|               title|total_ratings|
+--------------------+-------------+
|Shawshank Redempt...|         1547|
| Forrest Gump (1994)|         1544|
| Pulp Fiction (1994)|         1515|
|Silence of the La...|         1423|
|  Matrix, The (1999)|         1381|
|Star Wars: Episod...|         1307|
|Jurassic Park (1993)|         1204|
|Schindler's List ...|         1144|
|Star Wars: Episod...|         1117|
|   Fight Club (1999)|         1113|
+--------------------+-------------+
only showing top 10 rows


In [ ]:
from pyspark.sql.functions import avg

avg_rating_movies = ratings_movies.groupBy("title") \
    .agg(avg("rating").alias("avg_rating")) \
    .orderBy("avg_rating", ascending=False)

avg_rating_movies.show(10)


+--------------------+----------+
|               title|avg_rating|
+--------------------+----------+
|Can You Keep a Se...|       5.0|
|Immigrant, The (2...|       5.0|
|Mother's Heart (1...|       5.0|
|Gunbuster (Top wo...|       5.0|
|    Atrocious (2010)|       5.0|
|  I Am Divine (2013)|       5.0|
|A Song of Lisbon ...|       5.0|
|       Hatred (2016)|       5.0|
|    Lake City (2008)|       5.0|
|Classe américaine...|       5.0|
+--------------------+----------+
only showing top 10 rows


In [ ]:
ratings_movies.groupBy("userId") \
    .agg(count("rating").alias("ratings_count")) \
    .orderBy("ratings_count", ascending=False) \
    .show(10)


+------+-------------+
|userId|ratings_count|
+------+-------------+
|  2177|         4227|
|   548|         3212|
|  1748|         3193|
|   847|         2920|
|  2982|         2803|
|   997|         2371|
|  1920|         2328|
|  1977|         1907|
|   626|         1888|
|  1652|         1853|
+------+-------------+
only showing top 10 rows


In [ ]:
from pyspark.sql.functions import explode, split

genres_df = ratings_movies.withColumn(
    "genre",
    explode(split("genres", "\\|"))
)

genres_df.groupBy("genre") \
    .agg(avg("rating").alias("avg_rating")) \
    .orderBy("avg_rating", ascending=False) \
    .show()


+------------------+------------------+
|             genre|        avg_rating|
+------------------+------------------+
|         Film-Noir| 3.936503984063745|
|               War|3.8118888649580014|
|             Crime|3.7045695223975432|
|             Drama|3.6987082086048755|
|       Documentary| 3.688826916156891|
|           Mystery|3.6878705224192156|
|         Animation| 3.642255835792863|
|(no genres listed)|3.6292134831460676|
|              IMAX| 3.611305665900063|
|           Western|3.5908572731418147|
|           Musical|3.5782549265292176|
|           Romance|3.5582604926677743|
|          Thriller| 3.537729045792769|
|           Fantasy| 3.527473063710181|
|         Adventure|3.5270564191026192|
|            Sci-Fi| 3.495831361968211|
|            Action|3.4814108110906496|
|          Children|  3.45967618950793|
|            Comedy| 3.443148518961686|
|            Horror| 3.314023937512993|
+------------------+------------------+

